### 직접 테스트하고 싶은 거 확인하면서 보기

In [4]:
# Repeat node 예제
import numpy as np

D, N = 8, 7
x = np.random.randn(1, D)
# np.repeat으로 x를 한 줄씩 추가하는 것
y = np.repeat(x, N, axis=0) # row를 기준으로 복제하기
print(f'x size: {x.shape}')
print(f'y size: {y.shape}')

x size: (1, 8)
y size: (7, 8)


In [5]:
x

array([[-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361]])

In [6]:
# x가 복제되어 같은 값이 계속 들어감
y 

array([[-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361],
       [-1.34697905, -0.04839212, -0.31104008,  0.95931094,  0.34183142,
         0.44716615,  0.85842551, -1.82937361]])

In [9]:
dy = np.random.randn(N, D)
# keepdims = True : 차원 수 유지
dx = np.sum(dy, axis=0, keepdims = True) # row를 기준으로 sum하겠다는 것

# sum을 한다는 것은 순전파
# 큰 dimension에서 압축되어 작은 dimension으로 감

# repeat을 한다는 것은 역전파
# 작은 dimension에서 큰 dimension으로 복제해가며 늘리는 작업

In [11]:
# sum 노드 역전파

x = np.random.randn(N, D)
y = np.sum(y, axis=0, keepdims=True)

print(f'x.shape: {x.shape}')
print(f'y.shape: {y.shape}')

x.shape: (7, 8)
y.shape: (1, 8)


### Matmul
행렬의 곱셈 Matrix Multiply의 약자로 `MatMul`을 사용한다. 배치사이즈가 N이라고 할 때 MatMul 노드의 역전파는 다음과 같다.
![matmul](/Users/taghive/Documents/minah/git/ML_study/deep_learning/ch01-Neraul_Network/image/matmul.png)

In [12]:
import numpy as np

# Matmul 클래스 구현
# common/layer.py

class Matmul:
    def __init__(self, W) -> None:
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW # 깊은 복사
        return dx

- **np.zeros** : 0으로 가득찬 Array를 배출한다. 즉, 여기에는 튜플, int, 혹은 list의 값이 들어와야 한다 그렇게 되면 해당하는 shape으로 형태를 만들어준다음 Array를 return 한다.    
만약 여기에 np.zeros_like 처럼 변수를 넣어주면 오류가 나온다.
- **np.zeros_like** : 어떤 변수만큼의 사이즈인 0 으로 가득 찬 Array를 배출한다. 즉, 여기에는 변수가 들어와야한다.    
여기는 변수 말고도 그냥 [2,3,3] 이렇게 parameter 로 넣어줘도 되는데 이때는 단, 2,3,3 shape을 가진 array가 나오는 것이 아니라 [0, 0, 0] 인 numpy array 가 나온다.

In [13]:
# 얕은 복사 깊은 복사 차이
a = np.array([1,2,3])
b = np.array([2,3,4])

a = b

In [14]:
id(a) == id(b)

True

In [16]:
b

array([2, 3, 4])

In [17]:
a = np.array([1,2,3])
b = np.array([2,3,4])

a[...] = b

In [18]:
id(a) == id(b)

False